# DS542 Midterm Part-2

### Sophisticated CNN – ResNet18

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Training + Evaluation
# -----------------------------
def main():
    CONFIG = {
        "model": "ResNet18_Efficient",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 30,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Augmentation
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # -----------------------------
    # Load CIFAR-100
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define ResNet18 
    # -----------------------------
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 100)
    model = model.to(CONFIG["device"])

    # -----------------------------
    # Loss, Optimizer, Scheduler
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # WandB Tracking
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="ResNet18")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part2resnet18.pth")
            wandb.save("best_model_part2resnet18.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part2resnet18.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part2resnet18.csv", index=False)
    print("submission_ood_part2resnet18.csv created successfully.")

if __name__ == '__main__':
    main()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: samritha (samritha-boston-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 2/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part2resnet18.csv created successfully.


### Sophisticated CNN – DenseNet121

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })
        
    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "DenseNet121_Efficient",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 30,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model: DenseNet121 
    # -----------------------------
    model = models.densenet121(weights=None)
    model.classifier = nn.Linear(model.classifier.in_features, 100)
    model = model.to(CONFIG["device"])

    # -----------------------------
    # Loss, Optimizer, Scheduler
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B Setup
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="DenseNet121")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part2densenet121.pth")
            wandb.save("best_model_part2densenet121.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part2densenet121.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part2densenet121.csv", index=False)
    print("submission_ood_part2densenet121.csv created successfully.")

if __name__ == '__main__':
    main()


Epoch 1/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 2/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 3/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part2densenet121.csv created successfully.
